In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf


2025-07-30 10:00:46.479830: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 10:00:46.480237: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 10:00:46.482713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 10:00:46.489046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753857046.500190   10205 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753857046.50

In [2]:
def read_data(path):
    df=pd.read_csv('/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/HateSpeechDatasetBalanced.csv')
    return df